# Tutorial 1: Data Generation and Problem Instances

**WSmart+ Route Tutorial Series**

This tutorial covers how to generate problem instances for Vehicle Routing Problems (VRP) using WSmart+ Route. You'll learn:

1. The **TensorDict** data format used throughout the framework
2. Using **generators** to create problem instances on-the-fly
3. The **VRPInstanceBuilder** for batch dataset creation
4. Different **distribution types** for waste/prize values
5. **Dataset classes** for PyTorch DataLoader integration
6. **Visualizing** problem instances

**Prerequisites**: A working WSmart+ Route installation (`uv sync`)

**Next tutorial**: [02_environments.ipynb](02_environments.ipynb) - RL Environments and Problem Formulation

In [ ]:
# Environment setup
import os
import sys
import warnings

warnings.filterwarnings("ignore")

# Add project root to path (two levels up from notebooks/tutorials/)
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print(f"Project root: {PROJECT_ROOT}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from tensordict import TensorDict

# Reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")

---

## 1. Understanding TensorDict

WSmart+ Route uses **TensorDict** as the universal data format for problem instances. A TensorDict is a dictionary-like container for tensors that supports batching, indexing, and device management.

In [ ]:
# Create a simple TensorDict representing 4 VRP instances with 10 nodes
td = TensorDict(
    {
        "locs": torch.rand(4, 10, 2),       # Customer locations (x, y)
        "depot": torch.rand(4, 2),           # Depot location
        "demand": torch.rand(4, 10),         # Demand at each node
        "prize": torch.rand(4, 10),          # Prize for visiting each node
    },
    batch_size=[4],
)

print(f"Batch size: {td.batch_size}")
print(f"Keys: {list(td.keys())}")
print(f"Locations shape: {td['locs'].shape}  (batch, nodes, coords)")
print(f"Demand shape: {td['demand'].shape}     (batch, nodes)")
print()

# Indexing: get a single instance
instance = td[0]
print(f"Single instance keys: {list(instance.keys())}")
print(f"Single instance locs: {instance['locs'].shape}")
print()

# Slicing: get a sub-batch
sub_batch = td[:2]
print(f"Sub-batch size: {sub_batch.batch_size}")

---

## 2. Problem Instance Generators

Generators create batches of problem instances on-the-fly. They're the primary way to create training data in the RL pipeline.

### Available Generators

| Generator | Problem | Key Features |
|-----------|---------|-------------|
| `VRPPGenerator` | Vehicle Routing with Profits | Locations, waste, prizes, capacity |
| `WCVRPGenerator` | Waste Collection VRP | Locations, fill levels, overflow penalties |
| `SCWCVRPGenerator` | Stochastic WCVRP | Noisy observations of fill levels |

In [ ]:
from logic.src.envs.generators import VRPPGenerator, WCVRPGenerator, get_generator

# Create a VRPP generator for 20-node problems
gen = VRPPGenerator(
    num_loc=20,
    min_loc=0.0,
    max_loc=1.0,
    loc_distribution="uniform",
    waste_distribution="uniform",
    prize_distribution="uniform",
    capacity=1.0,
    depot_type="center",
)

# Generate a batch of 16 instances
td = gen(batch_size=16)

print("Generated TensorDict:")
print(f"  Batch size: {td.batch_size}")
print(f"  Keys: {list(td.keys())}")
print(f"  Locations shape: {td['locs'].shape}")
print(f"  Depot shape: {td['depot'].shape}")
print(f"  Waste shape: {td['waste'].shape}")
print(f"  Prize shape: {td['prize'].shape}")
print(f"  Capacity: {td['capacity'][0].item():.1f}")
print(f"  Max length: {td['max_length'][0].item():.2f}")

In [ ]:
def plot_instance(td, idx=0, title="VRPP Instance"):
    """Plot a single problem instance with depot, nodes, and prize values."""
    locs = td["locs"][idx].numpy()
    depot = td["depot"][idx].numpy()
    prizes = td["prize"][idx].numpy()

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Left: Node locations colored by prize
    ax = axes[0]
    scatter = ax.scatter(locs[:, 0], locs[:, 1], c=prizes, cmap="YlOrRd",
                         s=80, edgecolors="black", linewidth=0.5, zorder=2)
    ax.scatter(depot[0], depot[1], c="blue", s=200, marker="s",
               edgecolors="black", linewidth=1, zorder=3, label="Depot")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_title(f"{title} - Node Locations")
    ax.legend()
    plt.colorbar(scatter, ax=ax, label="Prize Value")

    # Right: Distribution of prizes and waste
    ax = axes[1]
    waste = td["waste"][idx].numpy()
    ax.hist(prizes, bins=15, alpha=0.6, label="Prize", color="green")
    ax.hist(waste, bins=15, alpha=0.6, label="Waste/Demand", color="orange")
    ax.set_xlabel("Value")
    ax.set_ylabel("Count")
    ax.set_title("Prize & Waste Distributions")
    ax.legend()

    plt.tight_layout()
    plt.show()

plot_instance(td, idx=0, title="VRPP (20 nodes, uniform)")

In [ ]:
# Compare different location distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
distributions = ["uniform", "normal", "clustered"]

for ax, dist in zip(axes, distributions):
    gen_dist = VRPPGenerator(num_loc=50, loc_distribution=dist)
    td_dist = gen_dist(batch_size=1)
    locs = td_dist["locs"][0].numpy()
    depot = td_dist["depot"][0].numpy()

    ax.scatter(locs[:, 0], locs[:, 1], s=30, alpha=0.8, label="Customers")
    ax.scatter(depot[0], depot[1], c="red", s=100, marker="s", label="Depot")
    ax.set_title(f"Location Distribution: {dist}")
    ax.set_xlim(-0.05, 1.05)
    ax.set_ylim(-0.05, 1.05)
    ax.set_aspect("equal")
    ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
# Waste Collection VRP generator
wc_gen = WCVRPGenerator(
    num_loc=20,
    min_fill=0.0,
    max_fill=1.0,
    fill_distribution="uniform",
    capacity=1.0,
)

td_wc = wc_gen(batch_size=16)

print("WCVRP TensorDict:")
print(f"  Keys: {list(td_wc.keys())}")
print(f"  Fill levels shape: {td_wc['demand'].shape}")
print(f"  Fill level range: [{td_wc['demand'].min():.3f}, {td_wc['demand'].max():.3f}]")

In [ ]:
from logic.src.envs import GENERATOR_REGISTRY, get_generator

print("Available generators:")
for name, gen_cls in GENERATOR_REGISTRY.items():
    print(f"  {name}: {gen_cls.__name__}")

# Use the factory function
gen_vrpp = get_generator("vrpp", num_loc=20)
gen_wcvrp = get_generator("wcvrp", num_loc=20)
print(f"\nVRPP generator: {type(gen_vrpp).__name__}")
print(f"WCVRP generator: {type(gen_wcvrp).__name__}")

---

## 3. VRPInstanceBuilder: Batch Dataset Creation

For generating larger datasets (training, validation, test), use the **VRPInstanceBuilder** which provides a fluent builder pattern with support for empirical and gamma distributions.

In [ ]:
from logic.src.data.builders import VRPInstanceBuilder

# Create builder and configure it
builder = VRPInstanceBuilder()
builder.set_dataset_size(32)
builder.set_problem_size(20)
builder.set_distribution("unif")
builder.set_problem_name("vrpp")
builder.set_num_days(1)

# Build as TensorDict
td_built = builder.build_td()

print("Built TensorDict:")
print(f"  Keys: {list(td_built.keys())}")
print(f"  Batch size: {td_built.batch_size}")
for key in td_built.keys():
    print(f"  {key}: shape={td_built[key].shape}, dtype={td_built[key].dtype}")

In [ ]:
# Compare different waste/prize distributions
dist_names = ["unif", "gamma1", "gamma2", "gamma3", "const"]

fig, axes = plt.subplots(1, len(dist_names), figsize=(18, 3.5))

for ax, dist in zip(axes, dist_names):
    builder = VRPInstanceBuilder()
    builder.set_dataset_size(100)
    builder.set_problem_size(20)
    builder.set_distribution(dist)
    builder.set_problem_name("vrpp")
    builder.set_num_days(1)

    td_d = builder.build_td()
    values = td_d["demand"].flatten().numpy()

    ax.hist(values, bins=30, alpha=0.7, color="steelblue", edgecolor="white")
    ax.set_title(f"Distribution: {dist}")
    ax.set_xlabel("Demand/Waste Value")
    ax.set_ylabel("Frequency")

plt.suptitle("Comparing Demand Distributions", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---

## 4. Dataset Classes and DataLoaders

WSmart+ Route provides dataset classes compatible with PyTorch's DataLoader for efficient batched training.

In [ ]:
from logic.src.data.datasets import GeneratorDataset, TensorDictDataset, tensordict_collate_fn
from torch.utils.data import DataLoader

# Option 1: From pre-generated TensorDict
td_data = gen(batch_size=128)
dataset = TensorDictDataset(td_data)
print(f"TensorDictDataset: {len(dataset)} instances")

# Create DataLoader with proper collation
loader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=tensordict_collate_fn,
)

# Iterate through one batch
batch = next(iter(loader))
print(f"Batch keys: {list(batch.keys())}")
print(f"Batch locs shape: {batch['locs'].shape}")

In [ ]:
# Option 2: GeneratorDataset generates data on-the-fly
gen_dataset = GeneratorDataset(gen, size=256)
print(f"GeneratorDataset: {len(gen_dataset)} instances")

gen_loader = DataLoader(
    gen_dataset,
    batch_size=32,
    collate_fn=tensordict_collate_fn,
)

batch = next(iter(gen_loader))
print(f"On-the-fly batch locs shape: {batch['locs'].shape}")

---

## 5. Saving and Loading Datasets

Datasets can be saved to disk for reproducible experiments.

In [ ]:
import tempfile

# Generate a dataset
td_save = gen(batch_size=64)

# Save to a temporary file
with tempfile.NamedTemporaryFile(suffix=".pt", delete=False) as f:
    save_path = f.name
    torch.save(td_save, save_path)
    print(f"Saved dataset to: {save_path}")
    print(f"File size: {os.path.getsize(save_path) / 1024:.1f} KB")

# Load it back
td_loaded = torch.load(save_path, weights_only=False)
print(f"\nLoaded dataset:")
print(f"  Batch size: {td_loaded.batch_size}")
print(f"  Keys: {list(td_loaded.keys())}")

# Verify equality
assert torch.allclose(td_save["locs"], td_loaded["locs"])
print("\nDatasets match!")

# Cleanup
os.unlink(save_path)

---

## 6. Putting It All Together: Generate a Complete Dataset

Let's create a full training/validation/test dataset split.

In [ ]:
def generate_dataset_split(gen, train_size=512, val_size=128, test_size=128, seed=42):
    """Generate train/val/test splits with different seeds."""
    torch.manual_seed(seed)
    train_data = gen(batch_size=train_size)

    torch.manual_seed(seed + 1)
    val_data = gen(batch_size=val_size)

    torch.manual_seed(seed + 2)
    test_data = gen(batch_size=test_size)

    return train_data, val_data, test_data


# Generate splits for VRPP with 20 nodes
gen_20 = VRPPGenerator(num_loc=20, loc_distribution="uniform")
train_data, val_data, test_data = generate_dataset_split(gen_20)

print("Dataset splits:")
print(f"  Train: {train_data.batch_size[0]} instances")
print(f"  Val:   {val_data.batch_size[0]} instances")
print(f"  Test:  {test_data.batch_size[0]} instances")

# Verify no overlap by checking first instance locations
assert not torch.allclose(train_data["locs"][0], val_data["locs"][0])
print("\nSplits are independent (no data leakage).")

In [ ]:
# Summary statistics of the generated data
def dataset_summary(td, name="Dataset"):
    """Print summary statistics for a TensorDict dataset."""
    print(f"\n{name}:")
    print(f"  Instances: {td.batch_size[0]}")
    print(f"  Nodes per instance: {td['locs'].shape[1]}")
    print(f"  Location range: [{td['locs'].min():.3f}, {td['locs'].max():.3f}]")
    if "prize" in td.keys():
        print(f"  Prize - mean: {td['prize'].mean():.3f}, std: {td['prize'].std():.3f}")
    if "waste" in td.keys():
        print(f"  Waste - mean: {td['waste'].mean():.3f}, std: {td['waste'].std():.3f}")
    if "demand" in td.keys():
        print(f"  Demand - mean: {td['demand'].mean():.3f}, std: {td['demand'].std():.3f}")


dataset_summary(train_data, "Training Set")
dataset_summary(val_data, "Validation Set")
dataset_summary(test_data, "Test Set")

---

## Summary

In this tutorial, you learned:

- **TensorDict** is the universal data format — a batched dictionary of tensors with keys like `locs`, `depot`, `waste`, `prize`, `capacity`
- **Generators** (`VRPPGenerator`, `WCVRPGenerator`) create instances on-the-fly with configurable distributions (`uniform`, `normal`, `clustered`)
- **VRPInstanceBuilder** provides batch dataset generation with additional distribution types (`gamma1-4`, `empirical`)
- **Dataset classes** (`TensorDictDataset`, `GeneratorDataset`) integrate with PyTorch DataLoaders
- Datasets can be **saved/loaded** with `torch.save`/`torch.load`

### Next Steps

Continue to **[Tutorial 2: RL Environments](02_environments.ipynb)** to learn how these problem instances are used in the reinforcement learning environment for training routing agents.